In [1]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import glob
from collections import Counter
import time

from vrp_eval import *
from mat_functs import *

import geopy.distance

In [2]:
def create_adj_mat(mat_dim, routing):
    adj_mat = np.zeros((mat_dim, mat_dim))
    
    for route in routing:
        for i in range(len(route)-1):
            adj_mat[route[i]][route[i+1]] += 1
            
    return adj_mat

In [3]:
def get_route_1MM(file):
    # returns full route of file
    # stop names; function used in transition matrix construction
    
    df = pd.read_csv(file)
    data = df[df['active']==1][['order','name']]
    
    num_of_routes = len(data[data['order']==1]) + len(data[data['name'].str.contains('SUMY')])
    
    # initialize; fix number of routes
    route_list = []
    for i in range(num_of_routes):
        route_list.append(['SUMY'])

    route_cnt = 0
    for i in range(len(data)):
        if 'SUMY' not in data.iloc[i]['name']:
            if data.iloc[i]['order'] != 1:
                route_list[route_cnt-1].append(data.iloc[i]['name'])
            else:
                route_cnt += 1
                route_list[route_cnt-1].append(data.iloc[i]['name'])
        else:
            route_cnt += 1

    # append 'SUMY' to end of each route
    for i in range(num_of_routes):
#         route_list[i].insert(0,'SUMY')
        route_list[i].append('SUMY')
#         route_list[i].append('SUMY')

    return route_list

In [4]:
# list of all files (train+test)
# file_list = [file for file in sorted(glob.glob("*.csv")) if 'SAMEDI' in file]
file_list = [file for file in sorted(glob.glob("*.csv"))]

In [5]:
# gather all stops visited in historical data
all_stops_set = set()
for instance in file_list:
    all_stops_set.update(set(flatten(get_route_1MM(instance))))
all_stops_list = list(all_stops_set)

all_stops_list.remove('SUMY')
all_stops_list.insert(0, 'SUMY')

all_stops_list

['SUMY',
 'FARM MEISER',
 'La Matelotte (Fish4you)',
 'DATS LIEGE SERAING',
 'TFCO SENEFFE',
 'OTAN/ NATO  PROMARITIMES',
 'FARM LLN',
 'TFCO AUD',
 'TFCO LH Hulpe',
 'TFCO LIEGE',
 'Les idees a la pele',
 'FARM ST CATH',
 'Sodexo SWIFT SOLVAY',
 'TFCO SABLON',
 'Parlement Flamand',
 'TFCO ANV VOLK',
 'OCEAN MAREE',
 'Actiris',
 'Sodexo La Plaine',
 'DATS GOSSELIES',
 'Sodexo CDR-CESE',
 'TFCO MA Malines',
 'Les ferme de nos pilifs',
 'LINDD',
 'Dats Waterloo',
 'Dats Anderlecht',
 'TFCO VDK',
 'Dats LEUVEN',
 'TRAVIE (SEL)',
 'Carfrig',
 'FARM BASCULE',
 "Bio c'Bon Stockel",
 'FARM FC-IXL',
 'TFCO JETte',
 'TFCO STOckel',
 'TFCO WGH',
 'Sodexo EUROCONTROL',
 'PROMARITIMES',
 'BON PLAISIR',
 'Little Morning',
 'BIA MARA ANVERS',
 'Residentie Winderkinds',
 'Residentie Arthur (VULPIA)',
 'Fish4you',
 'Residentie Brugsevaart',
 'FARM TONGRES',
 'TFCO WOO Waterloo',
 'TFCO ANV BRASSCHAT',
 'DATS GRIMBERGEN',
 'TFCO LEUVEN',
 'TFCO GENT',
 'NOS PILIFS',
 'DB RENT',
 'DATS MELLE',
 'FARM CO

In [6]:
coords_list = [('SUMY', 50.911043, 4.403273)]

for item in all_stops_list[1:]:
    for file in file_list:
        df = pd.read_csv(file)
        data = df[df['active']==1][['order','name', 'lat', 'lng']]
        
        if item in data.values:
            coords = (item, data[data['name'] == item]['lat'].values[0], data[data['name'] == item]['lng'].values[0])
            coords_list.append(coords)
            break

coords_list

[('SUMY', 50.911043, 4.403273),
 ('FARM MEISER', 50.85543, 4.3951399),
 ('La Matelotte (Fish4you)', 50.81378, 4.3485699),
 ('DATS LIEGE SERAING', 50.60829, 5.51841),
 ('TFCO SENEFFE', 50.505976, 4.267244),
 ('OTAN/ NATO  PROMARITIMES', 50.877383, 4.429926),
 ('FARM LLN', 50.6689, 4.61494),
 ('TFCO AUD', 50.82046, 4.40897),
 ('TFCO LH Hulpe', 50.73138, 4.4809),
 ('TFCO LIEGE', 50.62798, 5.56695),
 ('Les idees a la pele', 50.8636, 4.37478),
 ('FARM ST CATH', 50.85269, 4.3483599),
 ('Sodexo SWIFT SOLVAY', 50.7411129, 4.4822104),
 ('TFCO SABLON', 50.84155, 4.35337),
 ('Parlement Flamand', 50.848518, 4.368233),
 ('TFCO ANV VOLK', 51.21187, 4.39655),
 ('OCEAN MAREE', 50.83987, 4.32004),
 ('Actiris', 50.85132, 4.37023),
 ('Sodexo La Plaine', 50.81958, 4.39348),
 ('DATS GOSSELIES', 50.4686, 4.43143),
 ('Sodexo CDR-CESE', 50.840304, 4.378256),
 ('TFCO MA Malines', 51.02265, 4.47407),
 ('Les ferme de nos pilifs', 50.90368, 4.38881),
 ('LINDD', 51.09705, 3.75923),
 ('Dats Waterloo', 50.69076, 4.3

In [7]:
# create full distance matrix
mat_dim = len(all_stops_list)
full_dist_mat = np.zeros((mat_dim, mat_dim))

for i in range(mat_dim):
    for j in range(mat_dim):
        full_dist_mat[i][j] = geopy.distance.distance((coords_list[i][1], coords_list[i][2]), 
                                                      (coords_list[j][1], coords_list[j][2])).km
        
full_dist_mat[0]

array([ 0.        ,  6.213145  , 11.48509099, 85.58757301, 46.07378917,
        4.18785538, 30.79564623, 10.08494792, 20.72162969, 87.92243261,
        5.64585265,  7.55478867, 19.70509673,  8.49130532,  7.37988244,
       33.47015775,  9.84940372,  7.03915979, 10.19819591, 49.25859064,
        8.06401417, 13.37502776,  1.30606877, 49.71713807, 25.10545093,
       11.63298407, 11.21484719, 20.28860498, 10.29440379, 13.1830451 ,
       11.05337205,  8.9642191 ,  8.94983349,  6.91316688,  9.11207395,
        7.44079978,  3.94225541,  7.63783798,  5.83194266, 32.49663508,
       34.47087225, 14.492475  , 32.76984353, 11.48509099, 53.00900652,
        7.77939698, 21.18738224, 42.47059874,  1.9667372 , 21.15261411,
       49.87273957,  1.30606877, 16.67371032, 45.85722064, 11.32788492,
       15.76958651, 39.77617876,  9.74359101,  3.26075935, 15.44084235,
       80.50007195, 63.30338787, 20.33791955, 10.26839292, 20.33791955,
       13.12322912, 26.28563587, 15.92224246,  7.40116774, 10.40

In [8]:
# # create full proba mat
# mat_dim = len(all_stops_list)
# full_proba_mat = np.zeros((mat_dim, mat_dim))

# # initialize with probabilities from 1st training instance
# for file in [file_list[0]]:
    
#     routing = get_route_1MM(file)
#     for route in routing:
#         for i in range(len(route)-1):
#             full_proba_mat[all_stops_list.index(route[i])][all_stops_list.index(route[i+1])] += 1
            
# full_proba_mat[0]

# initialize with distance matrix

full_proba_mat = full_dist_mat

In [9]:
# # proba mat transformations
# full_proba_mat = full_proba_mat + 1 # smoothing
# full_proba_mat = full_proba_mat/full_proba_mat.sum(axis=1, keepdims=True) # normalize
# # full_proba_mat = -np.log(full_proba_mat) # negative log

# full_proba_mat[0]

# proba mat transformations IF USING DIST_MAT AS INIT
full_proba_mat = full_proba_mat + 1 # smoothing
full_proba_mat = 1/full_proba_mat
full_proba_mat = full_proba_mat/full_proba_mat.sum(axis=1, keepdims=True) # normalize
# full_proba_mat = -np.log(full_proba_mat) # negative log

full_proba_mat[0]

array([0.13271363, 0.01839886, 0.01062977, 0.00153271, 0.00281927,
       0.0255816 , 0.00417396, 0.01197242, 0.00610975, 0.00149247,
       0.01996939, 0.01551337, 0.00640971, 0.01398265, 0.01583717,
       0.0038501 , 0.01223234, 0.0165084 , 0.01185134, 0.00264062,
       0.01464182, 0.00923223, 0.05754973, 0.00261674, 0.00508375,
       0.01050533, 0.01086494, 0.00623402, 0.01175039, 0.0093572 ,
       0.0110105 , 0.01331902, 0.01333828, 0.01677124, 0.01312427,
       0.01572287, 0.02685285, 0.01536422, 0.01942546, 0.003962  ,
       0.00374148, 0.00856633, 0.00392995, 0.01062977, 0.00245725,
       0.01511649, 0.00598149, 0.00305295, 0.04473387, 0.00599088,
       0.00260874, 0.05754973, 0.0075091 , 0.0028323 , 0.01076532,
       0.00791395, 0.00325469, 0.01235282, 0.03114788, 0.00807219,
       0.00162839, 0.00206387, 0.00621961, 0.01177751, 0.00621961,
       0.00939683, 0.00486386, 0.00784256, 0.01579705, 0.01163294,
       0.03504872, 0.00276402, 0.00496984, 0.01062164])

### Train

In [10]:
n_test = 49

for epoch in range(3):

    for current_inst in file_list[1:-n_test]:

        actual = convert(get_route_1MM(current_inst), all_stops_list)

        # relevant stops of the current instance
        current_idx_list = list(set(flatten(actual)))

        # proba mat for current instance
        proba_matrix = full_proba_mat[np.ix_(current_idx_list, current_idx_list)]

        n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
        N = [i for i in range(1, n+1)]
        rt_count = len(actual)

        Q = n

#         day = current_inst.split(" ")[1].split('.')[0]
    
#         stop_list = []
#         for i in range(len(current_idx_list)):
#             stop_list.append(all_stops_list[current_idx_list[i]])
        
#         if day == 'MARDI':
#             Q = 14
#         elif day == 'SAMEDI':
#             Q = 25 # 20?
#         else:
#             Q = 13

#         # add demands
#         cap = []
        
#         for i in range(len(stop_list)):
#             if any(['DATS' in stop_list[i], 'Dats' in stop_list[i]]):
#                 cap.append(1)
#             elif any(['TFCO LH' in stop_list[i], 'TFCO WOO' in stop_list[i], 'TFCO IXL' in stop_list[i]]):
#                 cap.append(4)
#             elif 'ST CATH' in stop_list[i]:
#                 if day == 'DIMANCHE' or day == 'LUNDI':
#                     cap.append(12)
#                 else:
#                     cap.append(10)
#             elif any(['TONGRES' in stop_list[i], 'HANKAR' in stop_list[i], 'TFCO OVErijse' in stop_list[i], 'TFCO VDK' in stop_list[i]],):
#                 cap.append(3)
#             else:
#                 cap.append(2)

        V = [0] + N
#         q = dict(zip(V, cap))
        q = {i : 1 for i in N} # dictionary of demands

        # create set of arcs
        A = [(i,j) for i in V for j in V if i!=j]
     
        # solve using CPLEX
        mdl = Model('CVRP')
        
        #         # gap tolerance
        #         mdl.parameters.mip.tolerances.mipgap=0.02;
        #         mdl.parameters.emphasis.mip=3;
        #         mdl.parameters.mip.strategy.search=1;

        x = mdl.binary_var_dict(A, name='x')
        u = mdl.continuous_var_dict(N, ub=Q, name='u')

        # mdl.minimize(mdl.sum((proba_matrix[i,j])*x[i,j] for i,j in A))
        mdl.minimize(mdl.sum(-np.log(proba_matrix[i,j])*x[i,j] for i,j in A))

        # constraints
        mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
        mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
        mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
        mdl.add_constraints(u[i]>=q[i] for i in N)

        # fix number of routes
        mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
        mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

        # set time limit
        #         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
        mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)

        start = time.time()

        # set (log_output=True) to display solution
        solution = mdl.solve()
        #         solution = mdl.solve(log_output=True)
        #         print("gap:",mdl.MIP.get_mip_relative_gap())
        #         print("gap:",mdl.get_solve_details().mip_relative_gap)
        gap = mdl.get_solve_details().mip_relative_gap

        end = time.time()

        sol_time = end-start

        print("solution time:", sol_time)

#         print('mdl.objective_value =', mdl.objective_value)

        active_arcs = [a for a in A if x[a].solution_value>0.9]
        start_arcs = [a for a in active_arcs if a[0]==0]

        res_plan = []
        for arc in start_arcs:    
            res_route = [0,arc[1]]
            while res_route[-1]!=0:
                for a in active_arcs:
                    if res_route[-1]==a[0]:
                        res_route.append(a[1])
            res_plan.append(res_route)

        d = dict(zip(V, current_idx_list))

        new_res_plan = []

        for route in res_plan:
            new_route = []
            for s in route:
                new_route.append(d[s])
            new_res_plan.append(new_route)

        actual_mat = create_adj_mat(len(full_proba_mat), actual)
        pred_mat = create_adj_mat(len(full_proba_mat), new_res_plan)

        full_proba_mat = full_proba_mat + 0.00002*(actual_mat-pred_mat)
#         full_proba_mat[full_proba_mat < 0] = 0

solution time: 0.6560747623443604
solution time: 0.40990161895751953
solution time: 0.31360316276550293
solution time: 1.2054708003997803
solution time: 0.9135053157806396
solution time: 0.23428893089294434
solution time: 0.22934675216674805
solution time: 0.4911043643951416
solution time: 1.0068292617797852
solution time: 1.0023210048675537
solution time: 1.1247360706329346
solution time: 0.7861766815185547
solution time: 0.13104557991027832
solution time: 0.19652152061462402
solution time: 0.21395254135131836
solution time: 1.2497141361236572
solution time: 0.2030799388885498
solution time: 0.1802208423614502
solution time: 0.39053773880004883
solution time: 1.0079503059387207
solution time: 1.2653350830078125
solution time: 0.20308136940002441
solution time: 1.1871988773345947
solution time: 0.5467519760131836
solution time: 0.1978607177734375
solution time: 0.7219977378845215
solution time: 0.8591773509979248
solution time: 1.1091201305389404
solution time: 0.18742871284484863
solu

solution time: 20.011011838912964
solution time: 1.1601240634918213
solution time: 0.46901559829711914
solution time: 1.421548843383789
solution time: 1.3277809619903564
solution time: 1.4003231525421143
solution time: 1.1848211288452148
solution time: 1.3691697120666504
solution time: 1.4300546646118164
solution time: 0.46498584747314453
solution time: 4.34899640083313
solution time: 1.2397756576538086
solution time: 2.020536184310913
solution time: 1.1452641487121582
solution time: 2.6584134101867676
solution time: 0.8784584999084473
solution time: 0.4044010639190674
solution time: 1.8370065689086914
solution time: 1.479719638824463
solution time: 2.20064377784729
solution time: 1.9213924407958984
solution time: 1.4527907371520996
solution time: 1.619140863418579
solution time: 1.6246263980865479
solution time: 1.343416690826416
solution time: 0.8904223442077637
solution time: 0.48772668838500977
solution time: 1.0935006141662598
solution time: 2.405695915222168
solution time: 1.4840

### Test

In [11]:
# initialize df of results
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time', 'Epoch'])

for current_inst in file_list[-n_test:]:
    
    actual = convert(get_route(current_inst), all_stops_list)

    # relevant stops of the current instance
    current_idx_list = list(set(flatten(actual)))
    
    # proba mat for current instance
    init_proba_mat = full_proba_mat[np.ix_(current_idx_list, current_idx_list)]
    
    n = len(set(flatten(actual)))-1 # len(stop_list)-1

    N = [i for i in range(1, n+1)]
    rt_count = len(actual)

    Q = n
    
#     day = current_inst.split(" ")[1].split('.')[0]

#     if day == 'MARDI':
#         Q = 14
#     elif day == 'SAMEDI':
#         Q = 25 # 20?
#     else:
#         Q = 13

#     # add demands
#     cap = []

#     for i in range(len(stop_list)):
#         if any(['DATS' in stop_list[i], 'Dats' in stop_list[i]]):
#             cap.append(1)
#         elif any(['TFCO LH' in stop_list[i], 'TFCO WOO' in stop_list[i], 'TFCO IXL' in stop_list[i]]):
#             cap.append(4)
#         elif 'ST CATH' in stop_list[i]:
#             if day == 'DIMANCHE' or day == 'LUNDI':
#                 cap.append(12)
#             else:
#                 cap.append(10)
#         elif any(['TONGRES' in stop_list[i], 'HANKAR' in stop_list[i], 'TFCO OVErijse' in stop_list[i], 'TFCO VDK' in stop_list[i]],):
#             cap.append(3)
#         else:
#             cap.append(2)

    V = [0] + N
#         q = dict(zip(V, cap))
    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]

    # solve using CPLEX
    mdl = Model('CVRP')

    #         # gap tolerance
    #         mdl.parameters.mip.tolerances.mipgap=0.02;
    #         mdl.parameters.emphasis.mip=3;
    #         mdl.parameters.mip.strategy.search=1;

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u')

    mdl.minimize(mdl.sum(-np.log(init_proba_mat[i,j])*x[i,j] for i,j in A))

    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
    #         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)

    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
    #         solution = mdl.solve(log_output=True)
    #         print("gap:",mdl.MIP.get_mip_relative_gap())
    #         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap

    end = time.time()

    sol_time = end-start

    print("solution time:", sol_time)

    print('mdl.objective_value =', mdl.objective_value)
    
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)

    d = dict(zip(current_idx_list, V))
    #         print(idx_list)

    new_actual = []

    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)

    actual_for_comp = [route[1:-1] for route in new_actual]
    
    actual_mat = create_adj_mat(len(full_proba_mat), actual)
    pred_mat = create_adj_mat(len(full_proba_mat), new_res_plan)
    
    full_proba_mat = full_proba_mat + 0.00002*(actual_mat-pred_mat)
#     full_proba_mat[full_proba_mat < 0] = 0
    
    results_df.loc[len(results_df)] = [current_inst.split(" ")[0], 
                                       actual_for_comp, 
                                       res_plan, gap,
                                       eval_sd(actual_for_comp,res_plan)[1], 
                                       eval_sd(actual_for_comp,res_plan)[2],
                                       eval_ad(actual_for_comp,res_plan)[1], 
                                       eval_ad(actual_for_comp,res_plan)[2], 
                                       gap, sol_time, epoch]

solution time: 0.10871577262878418
mdl.objective_value = 123.62475565145475
solution time: 0.09369254112243652
mdl.objective_value = 75.36972738620325
solution time: 0.21870088577270508
mdl.objective_value = 132.39399950597593
solution time: 0.27222633361816406
mdl.objective_value = 114.44211061550808
solution time: 0.4790477752685547
mdl.objective_value = 116.70401406676356
solution time: 0.21870017051696777
mdl.objective_value = 134.05008230536018
solution time: 0.1828601360321045
mdl.objective_value = 121.14444431847656
solution time: 0.135331392288208
mdl.objective_value = 71.95040762030514
solution time: 0.22237372398376465
mdl.objective_value = 132.38394786348232
solution time: 0.10011553764343262
mdl.objective_value = 106.6601348667765
solution time: 0.15619444847106934
mdl.objective_value = 134.00449858652405
solution time: 0.18398356437683105
mdl.objective_value = 136.39574024468908
solution time: 0.1234588623046875
mdl.objective_value = 129.13721814017526
solution time: 0.071

In [12]:
results_df

,Date,Target,Solution,Status,SD,%SD,AD,%AD,Gap,Time,Epoch
0,190208,"[[0, 18, 0], [0, 11, 10, 18, 0], [0, 4, 2, 24,...","[[0, 1, 0], [0, 4, 25, 20, 24, 2, 5, 21, 8, 16...",0.000096,17,0.653846,20,0.625000,0.000096,0.108716,2
1,190209,"[[0, 1, 10, 17, 14, 2, 8, 3, 15, 4, 0], [0, 5,...","[[0, 5, 2, 15, 4, 8, 3, 9, 16, 12, 13, 11, 6, ...",0.000000,7,0.388889,11,0.578947,0.000000,0.093693,2
2,190210,"[[0, 28, 14, 4, 26, 22, 27, 0], [0, 17, 21, 0]...","[[0, 1, 0], [0, 6, 0], [0, 12, 7, 28, 9, 0], [...",0.000093,18,0.620690,25,0.714286,0.000093,0.218701,2
3,190211,"[[0, 16, 0], [0, 10, 9, 0], [0, 3, 1, 22, 18, ...","[[0, 4, 19, 7, 15, 3, 1, 22, 18, 23, 0], [0, 5...",0.000093,14,0.583333,14,0.482759,0.000093,0.272226,2
4,190212,"[[0, 18, 26, 0], [0, 11, 10, 18, 4, 2, 24, 20,...","[[0, 1, 0], [0, 11, 7, 6, 0], [0, 12, 26, 0], ...",0.000015,17,0.629630,19,0.612903,0.000015,0.479048,2
5,190214,"[[0, 18, 26, 0], [0, 11, 10, 18, 0], [0, 4, 2,...","[[0, 1, 0], [0, 6, 0], [0, 11, 7, 26, 0], [0, ...",0.000100,20,0.714286,18,0.514286,0.000100,0.218700,2
6,190215,"[[0, 12, 11, 18, 0], [0, 4, 2, 24, 20, 25, 0],...","[[0, 1, 0], [0, 9, 0], [0, 12, 7, 6, 0], [0, 1...",0.000081,16,0.615385,21,0.677419,0.000081,0.182860,2
7,190216,"[[0, 1, 10, 16, 13, 2, 8, 3, 14, 4, 0], [0, 5,...","[[0, 9, 15, 12, 11, 8, 3, 4, 14, 2, 5, 7, 16, ...",0.000000,7,0.411765,11,0.611111,0.000000,0.135331,2
8,190217,"[[0, 21, 15, 7, 4, 28, 24, 29, 0], [0, 18, 23,...","[[0, 1, 0], [0, 13, 8, 6, 0], [0, 14, 10, 0], ...",0.000000,19,0.633333,26,0.722222,0.000000,0.222374,2
9,190218,"[[0, 10, 9, 0], [0, 3, 1, 22, 18, 23, 0], [0, ...","[[0, 10, 6, 5, 0], [0, 11, 0], [0, 12, 18, 23,...",0.000000,15,0.625000,18,0.642857,0.000000,0.100116,2


In [13]:
results_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_5932\3509099144.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results_df.mean()


Date      3.881800e+291
Status     5.035683e-05
SD         1.148980e+01
%SD        4.494986e-01
AD         1.702041e+01
%AD        5.577851e-01
Gap        5.035683e-05
Time       1.424737e-01
Epoch      2.000000e+00
dtype: float64

In [15]:
full_proba_mat.max()

0.5221186797649411

In [16]:
results_df.to_csv('SOP_Proba_ALL_00002_DistAsInit_ep3.csv')

In [ ]:
np.where(init_proba_mat < 0)

In [ ]:
actual = convert(get_route(current_inst), all_stops_list)

# relevant stops of the current instance
current_idx_list = list(set(flatten(actual)))

In [ ]:
# proba mat for current instance

init_proba_mat = full_proba_mat[np.ix_(current_idx_list, current_idx_list)]

In [ ]:
init_proba_mat[0]

In [ ]:
# proba mat transformations

init_proba_mat = init_proba_mat + 1 # smoothing
init_proba_mat = init_proba_mat/init_proba_mat.sum(axis=1, keepdims=True) # normalize
init_proba_mat = -np.log(init_proba_mat) # negative log

In [ ]:
init_proba_mat[0]

In [ ]:
# dist mat for current instance

init_dist_mat = full_dist_mat[np.ix_(current_idx_list, current_idx_list)]

In [ ]:
init_dist_mat[0]

In [ ]:
# dist mat transformations

init_dist_mat = init_dist_mat + 1 # smoothing
init_dist_mat = 1/init_dist_mat # invert
init_dist_mat = -np.log(init_dist_mat) # negative log

In [ ]:
init_dist_mat[0]

In [ ]:
actual = convert(get_route(current_inst), all_stops_list)

In [ ]:
actual

In [ ]:
n = len(set(flatten(actual)))-1 # len(stop_list)-1

In [ ]:
n

In [ ]:
N = [i for i in range(1, n+1)]
rt_count = len(actual)

Q = n

In [ ]:
V = [0] + N
#         q = dict(zip(V, cap))
q = {i : 1 for i in N} # dictionary of demands

# create set of arcs
A = [(i,j) for i in V for j in V if i!=j]

# solve using CPLEX
mdl = Model('CVRP')

#         # gap tolerance
#         mdl.parameters.mip.tolerances.mipgap=0.02;
#         mdl.parameters.emphasis.mip=3;
#         mdl.parameters.mip.strategy.search=1;

x = mdl.binary_var_dict(A, name='x')
u = mdl.continuous_var_dict(N, ub=Q, name='u')

In [ ]:
w_d = 1
w_p = 1

In [ ]:
proba_matrix = w_d*init_dist_mat + w_p*init_proba_mat 

In [ ]:
mdl.minimize(mdl.sum((proba_matrix[i,j])*x[i,j] for i,j in A))

In [ ]:
# constraints
mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
mdl.add_constraints(u[i]>=q[i] for i in N)

# fix number of routes
mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

# set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)

start = time.time()

# set (log_output=True) to display solution
solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
gap = mdl.get_solve_details().mip_relative_gap

end = time.time()

sol_time = end-start

print("solution time:", sol_time)

print('mdl.objective_value =', mdl.objective_value)

In [ ]:
active_arcs = [a for a in A if x[a].solution_value>0.9]
start_arcs = [a for a in active_arcs if a[0]==0]

In [ ]:
res_plan = []
for arc in start_arcs:    
    res_route = [0,arc[1]]
    while res_route[-1]!=0:
        for a in active_arcs:
            if res_route[-1]==a[0]:
                res_route.append(a[1])
    res_plan.append(res_route)

In [ ]:
res_plan

In [ ]:
actual

In [ ]:
d = dict(zip(current_idx_list, V))
#         print(idx_list)

new_actual = []

for route in actual:
    new_route = []
    for s in route:
        new_route.append(d[s])
    new_actual.append(new_route)
    
actual_for_comp = [route[1:-1] for route in new_actual]

In [ ]:
actual_for_comp

In [ ]:
eval_ad(actual_for_comp, res_plan)

In [ ]:
eval_sd(actual_for_comp, res_plan)

In [ ]:
actual_for_comp

In [ ]:
mat_dim = len(init_dist_mat)

In [ ]:
pred_adj = create_adj_mat(mat_dim, res_plan)
act_adj = create_adj_mat(mat_dim, actual_for_comp)

In [ ]:
w_d = w_d + 0.00001*(np.multiply(pred_adj, init_dist_mat).sum() - np.multiply(act_adj, init_dist_mat).sum()) 

In [ ]:
w_p = w_p + 0.00001*(np.multiply(pred_adj, init_proba_mat).sum() - np.multiply(act_adj, init_proba_mat).sum()) 